# SailGP Data Analyst Challenge

The aim is to test you python abilities. The challenge is to analyze the data provided and answer the questions below. You can use any library you want to help you with the analysis. The data is from the SailGP event in Auckland 2025. The data is in the 'DATA' folder.

There are various sources available.

The Boat Logs are in the 'Boat_Logs' folder. The data is in csv format and the columns are described in the 'Boat_Logs/Boat_Logs_Columns.csv' file.
The 'Course_Marks_2025-01-19.csv' file contains the mark positions and wind reading on the course for the whole day.

The Race_XML folder contains the xml files for each race that contains information on where the boundaries of the course are, the theoretical position of the marks and the target racecourse axis.

The 2025-01-19_man_summary.csv file contains the metrics from the manoeuvre summary for the day.
The 2025-01-19_straight_lines.csv file contains the metrics from the straight line summary for the day.

Both are derived from the boat logs.

The 2502 m8_APW_HSB2_HSRW.kph.csv file contains the polar data for the boats in that config.

## Requierements
- Chose at least 3 questions from the list below to answer.
- Python 3.8 or higher
- Notebook should be able to run without any errors from start to finish.
- Specify the libraries (imports) used in the notebook.
- Any comments to make the notebook self-explanatory and easy to follow would be appreciated.
- If you can't get to the end of a question, we would appreciate the code you have written so far and explain what you were trying to do.

## Further information:
- We usually use bokeh for visualizations. So any showcase of bokeh would be appreciated.
-

## Submitting the results.
It would be great if you could provide a jupyter notebook with the code and the results of the analysis. You can submit the results by sharing a link to a git repository.


### Imports and re-used functions
Free section to initialize the notebook with the necessary imports and functions that will be used in the notebook.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_bokeh 


## Question 1: Write a Python function that can take a compass direction (ie. TWD or Heading) and calculate an accurate mean value across a downsampled frequency. Eg. If TWD is at 1Hz, give me a 10s average.

In [119]:
import numpy as np

# This module is for demonstrating the compass direction conversion without using and importing the dataset. 

def circular_mean(angles):
 

   # Calculate the circular mean of a list of angles (in degrees).
    
    angles_rad = np.deg2rad(angles)  
    mean_cos = np.mean(np.cos(angles_rad))  
    mean_sin = np.mean(np.sin(angles_rad)) 
    mean_rad = np.arctan2(mean_sin, mean_cos) 
    mean_degree = np.rad2deg(mean_rad)  
    return mean_degree % 360  

def downsample_compass_data(data, original_frequency, target_frequency):
    
      #  Downsampling compass data and calculate the circular mean value for each interval.
      #  data: compass directions in degrees.
      #  original_frequency: Original sampling frequency in Hz.
      #  target_frequency: Targeted sampling frequency in Hz.
      # Returns: downsampled_data: downsampled mean compass directions.
    
    interval = int(original_frequency // target_frequency)  # Number of samples per downsampled interval
    downsampled_data = []
    
    for i in range(0, len(data), interval):
        chunk = data[i:i + interval]
        if len(chunk) > 0:
            mean_direction = circular_mean(chunk)
            downsampled_data.append(mean_direction)
    
    return downsampled_data
    
    if original_frequency <= target_frequency:
        raise ValueError("Target frequency should be lower than the original frequency.")
################################################################################################################################

#  If TWD is at 1Hz, give me a 10s average
if __name__ == "__main__":
    # Simulated 1Hz compass data (e.g., TWD or Heading)
     original_frequency = 1  # 1Hz
    
    #  compass directions examples using data_AUS csv file column K TWD_SGP_deg first 10 cells
     data = [66.96,53.63,54.16,54.68,55.73,56.69,57.57,58.39,59.15]  
   
    # Downsample to 0.1Hz 10s average
     target_frequency = 0.1
    
     downsampled_data = downsample_compass_data(data, original_frequency, target_frequency) # output downsampled data
    
     print("Downsampled Actual Mean Value Compass Data:", downsampled_data)





Downsampled Actual Mean Value Compass Data: [57.43575727958426]


## Question 2: Given a course XML and a timeseries of boat Lat/Lon values, calculate a VMC column for the same timeseries.


In [75]:
import pandas as pd
from bs4 import BeautifulSoup
from geopy.distance import geodesic

def calculate_vmc(lat1, lon1, lat2, lon2, speed, cog):
   
    #Calculate Velocity Made Good (VMC) towards a target point.
    
    # lat1: Current latitude
    # lon1: Current longitude
    # lat2: Target latitude
    # lon2: Target longitude
    # speed: Boat speed in km/h
    # cog: Course over ground in degrees
    # return: VMC in km/h
    
    current_point = (lat1, lon1)
    target_point = (lat2, lon2)
   # bearing_to_target = geodesic(current_point, target_point).initial_bearing
    #angle_diff = abs(cog - bearing_to_target)
    #vmc = speed * abs(cos(radians(angle_diff)))
   # return vmc
    from geopy.distance import geodesic, great_circle
    from geopy.point import Point
##########################################################################################################
# Calculates the initial bearing between two GPS coordinates.
def initial_bearing(lat1, lon1, lat2, lon2):
    
    import math
    
    delta_lon = lon2 - lon1
    x = math.sin(math.radians(delta_lon)) * math.cos(math.radians(lat2))
    y = (math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) -
         math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(delta_lon)))
    bearing = math.atan2(x, y)
    return (math.degrees(bearing) + 360) % 360
##############################################################################################################
    #Calculates the Velocity Made Good (VMC).
def calculate_vmc(lat1, lon1, lat2, lon2, speed, cog):
    bearing_to_target = initial_bearing(lat1, lon1, lat2, lon2)
    angle_diff = abs(cog - bearing_to_target)
    vmc = speed * abs(math.cos(math.radians(angle_diff)))
    return vmc
##############################################################################################################    
# Extract course marks from xml file
def import_and_process_data(xml_file, csv_file):
    # Parse the XML file
    with open(xml_file, 'r') as file:
        soup = BeautifulSoup(file, 'xml')
    marks = []
    for mark in soup.find_all('Mark'):
        marks.append({
            'Name': mark['Name'],
            'Lat': float(mark['TargetLat']),
            'Lon': float(mark['TargetLng'])
        })
    
    # Read the boat log AUS boat CSV file
    df = pd.read_csv(csv_file)
    
    # Assuming the first mark is the target for VMC calculation
    target_mark = marks[0]
    target_lat = target_mark['Lat']
    target_lon = target_mark['Lon']
    
    # Calculate VMC for each row in the dataframe
    df['VMC'] = df.apply(lambda row: calculate_vmc(
        row['LATITUDE_GPS_unk'], row['LONGITUDE_GPS_unk'],
        target_lat, target_lon,
        row['GPS_SOG_km_h_1'], row['GPS_COG_deg']
    ), axis=1)
    
    return df
################################################################################################################
# dataset import from first XML file and AUS Boat log data
xml_file = 'Data/Race_XMLs/25011905_03-13-55.xml'
csv_file =  'Data/Boat_logs/data_AUS.csv'
processed_data = import_and_process_data(xml_file, csv_file)
print(processed_data[['DATETIME', 'LATITUDE_GPS_unk', 'LONGITUDE_GPS_unk', 'VMC']])


                 DATETIME  LATITUDE_GPS_unk  LONGITUDE_GPS_unk        VMC
0     2025-01-19 02:56:08        -36.836543         174.767222  32.729371
1     2025-01-19 03:06:00        -36.833989         174.761755  13.501404
2     2025-01-19 03:06:01        -36.834000         174.761797  14.547035
3     2025-01-19 03:06:02        -36.834013         174.761841  15.994771
4     2025-01-19 03:06:03        -36.834029         174.761889  16.188467
...                   ...               ...                ...        ...
2336  2025-01-19 04:22:41        -36.835905         174.758753  38.101930
2337  2025-01-19 04:22:42        -36.835920         174.758874  35.032979
2338  2025-01-19 04:22:43        -36.835928         174.758987  32.127457
2339  2025-01-19 04:22:44        -36.835931         174.759088  28.749217
2340  2025-01-19 04:22:45        -36.835929         174.759177  25.226032

[2341 rows x 4 columns]


## Question 3: Verify and comment on the boats calibration. If possible propose a post-calibrated set of wind numbers and a potential calibration table.


## Question 4: Given a timeseries of Lat/Lon positions and a course XML, in a Python notebook, calculate a Distance to Leader metric for each boat.

## Question 5: Given a course XML, along with a wind speed and direction and a polar, calculate the minimum number of tacks or gybes for each leg of the course and each gate mark on the leg.

In [161]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from geopy.distance import geodesic
import math

def parse_course_xml(xml_file):
    #Parses the course XML file and extracts waypoints.
    with open(xml_file, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "xml")

    waypoints = []
    for corner in soup.find_all("Corner"):
        compound_mark_id = corner["CompoundMarkID"]
        mark = soup.find("CompoundMark", {"CompoundMarkID": compound_mark_id})
        for mark_seq in mark.find_all("Mark"):
            lat, lng = float(mark_seq["TargetLat"]), float(mark_seq["TargetLng"])
            waypoints.append((lat, lng))

    return waypoints

def parse_polar_csv(polar_file):
   #import the boat polar CSV file into a lookup table.
    df = pd.read_csv(polar_file)
    df.set_index(df.columns[0], inplace=True)
    df.columns = df.columns.astype(float)
    return df
#Calculates the minimum tacks or gybes needed for a leg
def calculate_tacks_or_gybes(start, end, True_wind_direction, polar_df):
    start_lat, start_lng = start
    end_lat, end_lng = end

    # Calculate course bearing (degrees)
    bearing = calculate_bearing(start_lat, start_lng, end_lat, end_lng)
    
    # Determine if upwind or downwind
    angle_to_wind = abs(True_wind_direction - bearing) % 360
    if angle_to_wind > 180:
        angle_to_wind = 360 - angle_to_wind
    
    # Get VMG angles from polar data, might be need to optimise this as the calculation is a bit simple
    best_upwind_angle, best_downwind_angle = get_optimal_angles(polar_df)
    
    if angle_to_wind < 90:  # Upwind
        optimal_angle = best_upwind_angle
    elif angle_to_wind > 90:  # Downwind
        optimal_angle = best_downwind_angle
    else:
        return 0  # No tacks/gybes needed if sailing perpendicular to the wind

    # Calculate minimum tacks/gybes required
    min_turns = calculate_turns(bearing, optimal_angle)
    
    return min_turns

def calculate_bearing(lat1, lon1, lat2, lon2):
    #Computes the bearing from (lat1, lon1) to (lat2, lon2).
    lat1, lat2 = map(math.radians, [lat1, lat2])
    diff_lon = math.radians(lon2 - lon1)

    x = math.sin(diff_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(diff_lon))
    bearing = math.degrees(math.atan2(x, y))
    return (bearing + 360) % 360

def get_optimal_angles(polar_df):
    #look up the best upwind and downwind VMG angles from the polar table
    twa_values = polar_df.index.astype(float)
    
    upwind_angles = twa_values[twa_values < 90]
    downwind_angles = twa_values[twa_values > 90]
    
    best_upwind_angle = upwind_angles[np.argmax(polar_df.loc[upwind_angles].max(axis=1))]
    best_downwind_angle = downwind_angles[np.argmax(polar_df.loc[downwind_angles].max(axis=1))]

    return best_upwind_angle, best_downwind_angle

def calculate_turns(bearing, optimal_angle):
    #Computes the minimum number of tacks or gybes required
    angle_diff = 180 - optimal_angle * 2  # Angle covered per tack/gybe
    return max(0, math.ceil(abs(bearing - 180) / angle_diff))

def analyze_course(xml_file, polar_file, wind_direction):
    #Analyzes the course and calculates the required tacks/gybes for each leg.
    waypoints = parse_course_xml(xml_file)
    polar_df = parse_polar_csv(polar_file)

    results = []
    for i in range(len(waypoints) - 1):
        start, end = waypoints[i], waypoints[i + 1]
        turns = calculate_tacks_or_gybes(start, end, True_wind_direction, polar_df)
        results.append((start, end, turns))

    return results

# extract the course xml file and boat polar file
xml_file = 'Data/Race_XMLs/25011905_03-13-55.xml'
polar_file = 'Data/2502 m8_APW_HSB2_HSRW.kph.csv'
True_wind_direction = 40  #pre-set the TWA variables

results = analyze_course(xml_file, polar_file, True_wind_direction)
for start, end, turns in results:
    print(f"Leg from {start} to {end}: {turns} tacks/gybes required")

    # result is a bit odd with some legs require 0 tacks, need to examine the course xml file to understand the course map


Leg from (-36.834644, 174.769757) to (-36.83533, 174.767562): 0 tacks/gybes required
Leg from (-36.83533, 174.767562) to (-36.829713, 174.76527): 9 tacks/gybes required
Leg from (-36.829713, 174.76527) to (-36.834715, 174.755873): 0 tacks/gybes required
Leg from (-36.834715, 174.755873) to (-36.832976, 174.753778): 7 tacks/gybes required
Leg from (-36.832976, 174.753778) to (-36.830518, 174.767965): 6 tacks/gybes required
Leg from (-36.830518, 174.767965) to (-36.82907, 174.767251): 8 tacks/gybes required
Leg from (-36.82907, 174.767251) to (-36.834715, 174.755873): 0 tacks/gybes required
Leg from (-36.834715, 174.755873) to (-36.832976, 174.753778): 7 tacks/gybes required
Leg from (-36.832976, 174.753778) to (-36.830518, 174.767965): 6 tacks/gybes required
Leg from (-36.830518, 174.767965) to (-36.82907, 174.767251): 8 tacks/gybes required
Leg from (-36.82907, 174.767251) to (-36.834715, 174.755873): 0 tacks/gybes required
Leg from (-36.834715, 174.755873) to (-36.836315, 174.758068):

## Question 6: Calculate a “tacked” set of variables depending on the tack of the boat, so that sailors don’t need to think about what tack they’re on when looking at measurements. And show the results in a visualisation.


In [157]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

# Function to calculate tacked variables based on the tack of the boat
def calculate_tacked_variables(wind_angle, boat_speed, tack):   
    
   # Parameters:
   # wind_angle (float): The angle of the wind relative to the boat (in degrees).
   # boat_speed (float): The speed of the boat (in knots).
   # tack (str): The tack of the boat, either 'port' or 'starboard'.
    
   # Returns:

    if tack not in ['port', 'starboard']:
        raise ValueError("Tack must be either 'port' or 'starboard'.")
    
    # Adjust wind angle based on tack
    if tack == 'port':
        tacked_wind_angle = wind_angle
    if tack == 'starboard':
        tacked_wind_angle = 360 - wind_angle
    
    # Calculate tacked boat speed (assuming no change in speed, just for demonstration)
    tacked_boat_speed = boat_speed
    
    # Calculate tacked variables (example: apparent wind speed)
    apparent_wind_speed = np.sqrt(boat_speed**2 + 10**2 - 2 * boat_speed * 10 * np.cos(np.radians(tacked_wind_angle)))
    
    return {
        'tacked_wind_angle': tacked_wind_angle,
        'tacked_boat_speed': tacked_boat_speed,
        'apparent_wind_speed': apparent_wind_speed
    }

# sample data for wind angle from 0 to 360 and bsp from 0 to 100 kph
wind_angles = np.linspace(0, 360, 72)
boat_speeds = np.linspace(0,100,72)
tack = 'port'
# tack = 'starboard'

# Calculate tacked variables for each wind angle and boat speed
tacked_variables = [calculate_tacked_variables(wa, bs, tack) for wa, bs in zip(wind_angles, boat_speeds)]

# Extract results for visualization
tacked_wind_angles = [tv['tacked_wind_angle'] for tv in tacked_variables]
apparent_wind_speeds = [tv['apparent_wind_speed'] for tv in tacked_variables]

# Create a Bokeh plot
output_notebook()  
p = figure(title="Tacked Variables Visualization", x_axis_label='Tacked Wind Angle (degrees)', y_axis_label='Apparent Wind Speed (knots)')
p.line(tacked_wind_angles, apparent_wind_speeds, legend_label="Apparent Wind Speed", line_width=2)

# Show the plot
show(p)

Loading BokehJS ...

## Question 7: Given a set of tacks (in CSV), and train a model to explain the key features of these tacks when optimizing for vmg. Show appropriate visualisations to explain your conclusions.

In [159]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6

# Function to import CSV, train a model, and visualize results using Bokeh
def analyze_tack_data_with_bokeh(csv_file):
    # 1: Import the CSV file
    data = pd.read_csv(csv_file)
    print("Data Head:")
    print(data.head())

    # 2: Preprocess the data

    if data.isnull().sum().any():
        print("Missing values found. Handling missing values...")
        data = data.dropna()  # or use data.fillna(...) for more sophisticated handling

    # Separate features and target (VMG)
    X = data.drop(columns=['theoretical_target_vmg'])  # Features
    y = data['theoretical_target_vmg']  # Target (VMG)

    # 3 Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 4 Train a Random Forest Regressor model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # 5 Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error: {mse}")

    # 6 Feature Importance Analysis
    feature_importances = model.feature_importances_
    feature_names = X.columns
    importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    print("\nFeature Importances:")
    print(importance_df)

    # 7 Permutation Importance for Robustness
    result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)
    perm_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': result.importances_mean})
    perm_importance_df = perm_importance_df.sort_values(by='Importance', ascending=False)

    print("\nPermutation Importances:")
    print(perm_importance_df)

    # 8 Visualize Feature Importances using Bokeh
    output_notebook()  # Render plots in a Jupyter notebook

    # Feature Importance Plot
    source = ColumnDataSource(importance_df)
    p1 = figure(
        x_range=importance_df['Feature'],
        title="Feature Importances for VMG Optimization",
        x_axis_label="Feature",
        y_axis_label="Importance",
        toolbar_location=None,
        tools=""
    )
    p1.vbar(
        x='Feature',
        top='Importance',
        width=0.9,
        source=source,
        line_color='white',
        fill_color=factor_cmap('Feature', palette=Spectral6, factors=importance_df['Feature'])
    )
    p1.add_tools(HoverTool(tooltips=[("Feature", "@Feature"), ("Importance", "@Importance")]))
    p1.xgrid.grid_line_color = None
    p1.y_range.start = 0

    # Permutation Importance Plot
    source_perm = ColumnDataSource(perm_importance_df)
    p2 = figure(
        x_range=perm_importance_df['Feature'],
        title="Permutation Importances for VMG Optimization",
        x_axis_label="Feature",
        y_axis_label="Importance",
        toolbar_location=None,
        tools=""
    )
    p2.vbar(
        x='Feature',
        top='Importance',
        width=0.9,
        source=source_perm,
        line_color='white',
        fill_color=factor_cmap('Feature', palette=Spectral6, factors=perm_importance_df['Feature'])
    )
    p2.add_tools(HoverTool(tooltips=[("Feature", "@Feature"), ("Importance", "@Importance")]))
    p2.xgrid.grid_line_color = None
    p2.y_range.start = 0

    # 9 Pairplot for Key Features (Top 3)
    top_features = importance_df['Feature'].head(3).tolist()
    for feature in top_features:
        p = figure(
            title=f"{feature} vs VMG",
            x_axis_label=feature,
            y_axis_label="VMG",
            tools="pan,wheel_zoom,box_zoom,reset,hover"
        )
        p.circle(data[feature], data['VMG'], size=8, alpha=0.6)
        show(p)

    # 10 Show all plots
    grid = gridplot([[p1, p2]])
    show(grid)

# extract data from csv file
csv_file = 'Data/2025-01-19_man_summary.csv' 
analyze_tack_data_with_bokeh(csv_file)

#couldn't fix the error in time

Data Head:
  BOAT HULL  WING_CONFIG_unk  MD4_SEL_DB_unk  MD4_SEL_RUD_unk  \
0  AUS  AUS             11.0             3.0              2.0   
1  AUS  AUS             11.0             3.0              2.0   
2  AUS  AUS             11.0             3.0              2.0   
3  AUS  AUS             11.0             3.0              2.0   
4  AUS  AUS             11.0             3.0              2.0   

                           DATETIME       TIME_LOCAL_unk  race  leg  type  \
0  2025-01-19T02:17:38.600000+00:00  2025-01-19 15:17:57     5    0  gybe   
1  2025-01-19T03:32:20.800000+00:00  2025-01-19 16:32:39     6    3  tack   
2  2025-01-19T03:33:20.200000+00:00  2025-01-19 16:33:39     6    4  gybe   
3  2025-01-19T03:34:17.200000+00:00  2025-01-19 16:34:36     6    4  tack   
4  2025-01-19T03:35:17.200000+00:00  2025-01-19 16:35:36     6    5  tack   

   ...  loss_vs_targ_vmg  drop_offset  drop_to_wind_axis  htw_bsp  entry_cant  \
0  ...       -205.029352         -0.5                6

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

## Question 8: Give insights on the racing on what made a team win or underperform in the race.